# Example of using the orm classes

Imports

In [1]:
from os import remove
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

In [2]:
remove('./tmp/test_orm_db.db')

Create the db engine and the database structure

In [3]:
engine = create_engine('sqlite:///tmp/test_orm_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2020-12-26 12:02:37,476 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-26 12:02:37,480 INFO sqlalchemy.engine.base.Engine ()
2020-12-26 12:02:37,484 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-26 12:02:37,485 INFO sqlalchemy.engine.base.Engine ()
2020-12-26 12:02:37,490 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-26 12:02:37,491 INFO sqlalchemy.engine.base.Engine ()
2020-12-26 12:02:37,494 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-26 12:02:37,494 INFO sqlalchemy.engine.base.Engine ()
2020-12-26 12:02:37,495 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2020-12-26 12:02:37,496 INFO sqlalchemy.engine.base.Engine ()
2020-12-26 12:02:37,497 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2020-12-26 12:02:37,500 INFO sqlalchemy.engine.base.Engine ()
2020-12-26

### Create the objects from las files

In [5]:
lexicon = Lexicon.default()

In [6]:
borehole_dict = {'F01':ROOT_DIR+'/data/test.las', 'F02':ROOT_DIR+'/data/test.las'} # boreholes to insert into the db

In [7]:
int_id = 0
bh_id = 0
pos_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
pos_dict = {}
x = [0., 20.]
y = [0., 120.]
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(BoreholeOrm(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        top = PositionOrm(id=pos_id, upper=interval.top.upper, middle=interval.top.middle,  
                          lower=interval.top.lower, x=x[bh_id], y=y[bh_id])
        base = PositionOrm(id=pos_id+1, upper=interval.base.upper, middle=interval.base.middle,  
                           lower=interval.base.lower, x=x[bh_id], y=y[bh_id])
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number, 
                          'top': top, 'base': base 
                         }})
        interval_number+=1
        int_id += 1
        pos_id += 2

    print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

{0: {'description': 'Anhydrite', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f481bc2d0b8>, 'base': <core.orm.PositionOrm object at 0x7f481bc2d4a8>}, 1: {'description': 'Sandstone, grey, vf-f', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f481bc2d198>, 'base': <core.orm.PositionOrm object at 0x7f481bc2d978>}, 2: {'description': 'Anhydrite', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f481bc2d278>, 'base': <core.orm.PositionOrm object at 0x7f481bc2d0f0>}, 3: {'description': 'Dolomite', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f481bc2d5c0>, 'base': <core.orm.PositionOrm object at 0x7f481bc2d518>}, 4: {'description': 'Anhydrite', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f481bc2d6d8>, 'base': <core.orm.PositionOrm object at 0x7f481bc2d550>}, 5: {'description': 'Sandstone, grey, vf-f', 'interval_number': 5, 'top': <core.orm.PositionOrm object at 0x7f481bc2d8d0>, 'base': <core.orm.Position

Create the session

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [9]:
p = Project(session)
p.add_components(components)

2020-12-26 12:02:37,889 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-26 12:02:37,893 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-26 12:02:37,896 INFO sqlalchemy.engine.base.Engine ()
2020-12-26 12:02:37,912 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2020-12-26 12:02:37,914 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2020-12-26 12:02:37,922 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-26 12:02:37,930 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-26 12:02:37,937 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-26 12:02:37,941 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [10]:
for bh in boreholes:
    p.add_borehole(bh)

2020-12-26 12:02:38,225 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-26 12:02:38,240 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-26 12:02:38,250 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2020-12-26 12:02:38,258 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 0.0, 0.0), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 0.0, 0.0)  ... displaying 10 of 50 total bound parameter sets ...  (48, 131.47647058823532, 131.476470588

2020-12-26 12:02:38,513 INFO sqlalchemy.engine.base.Engine (14,)
2020-12-26 12:02:38,520 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:38,523 INFO sqlalchemy.engine.base.Engine (15,)
2020-12-26 12:02:38,529 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:38,533 INFO sqlalchemy.engine.base.Engine (16,)
2020-12-26 12:02:38,541 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2020-12-26 12:02:38,753 INFO sqlalchemy.engine.base.Engine (35,)
2020-12-26 12:02:38,756 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:38,761 INFO sqlalchemy.engine.base.Engine (36,)
2020-12-26 12:02:38,763 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:38,768 INFO sqlalchemy.engine.base.Engine (37,)
2020-12-26 12:02:38,771 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2020-12-26 12:02:39,169 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:39,174 INFO sqlalchemy.engine.base.Engine (50,)
2020-12-26 12:02:39,179 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:39,180 INFO sqlalchemy.engine.base.Engine (51,)
2020-12-26 12:02:39,190 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Position

2020-12-26 12:02:39,365 INFO sqlalchemy.engine.base.Engine (70,)
2020-12-26 12:02:39,386 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:39,397 INFO sqlalchemy.engine.base.Engine (71,)
2020-12-26 12:02:39,404 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:39,419 INFO sqlalchemy.engine.base.Engine (72,)
2020-12-26 12:02:39,423 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2020-12-26 12:02:39,790 INFO sqlalchemy.engine.base.Engine (91,)
2020-12-26 12:02:39,802 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:39,808 INFO sqlalchemy.engine.base.Engine (92,)
2020-12-26 12:02:39,816 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-26 12:02:39,820 INFO sqlalchemy.engine.base.Engine (93,)
2020-12-26 12:02:39,846 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

In [11]:
p.boreholes[0].id

'F01'

In [12]:
p.commit()

2020-12-26 12:02:40,371 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
p.boreholes[0].intervals[0].description

2020-12-26 12:02:40,392 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-26 12:02:40,395 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2020-12-26 12:02:40,396 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-26 12:02:40,399 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2020-12-26 12:02:40,400 INFO sqlalchemy.engine.base.Engine ('F01',)


'Anhydrite'

In [14]:
session.close()

2020-12-26 12:02:40,436 INFO sqlalchemy.engine.base.Engine ROLLBACK
